In [10]:
import xgboost as xgb
import pandas as pd
import numpy as np

#train = pd.read_csv('../dataset/pre/train.csv')
labels = train["label"]

features = train.drop(['label','conversionTime'], axis=1)
data_dmat = xgb.DMatrix(data=features, label=labels)

params={"max_depth":5, "min_child_weight":2, "eta": 0.1, "subsamples":0.9, "colsample_bytree":0.8, "objective" : "binary:logistic", "eval_metric": "logloss", "seed": 2333}
rounds = 500

result = xgb.cv(params=params, dtrain=data_dmat, num_boost_round=rounds, early_stopping_rounds=20, as_pandas=True, seed=2333)
print result

     test-logloss-mean  test-logloss-std  train-logloss-mean  \
0             0.607529          0.000037            0.607529   
1             0.537504          0.000085            0.537503   
2             0.479259          0.000106            0.479256   
3             0.430216          0.000168            0.430207   
4             0.388558          0.000218            0.388545   
5             0.352889          0.000256            0.352873   
6             0.322185          0.000297            0.322163   
7             0.295587          0.000339            0.295558   
8             0.272505          0.000369            0.272469   
9             0.252418          0.000406            0.252374   
10            0.234881          0.000463            0.234827   
11            0.219553          0.000505            0.219493   
12            0.206158          0.000550            0.206088   
13            0.194415          0.000567            0.194330   
14            0.184111          0.000627

In [1]:
import pandas as pd

In [2]:
test = pd.read_csv('../dataset/pre/test.csv')
train = pd.read_csv('../dataset/pre/train.csv')

In [3]:
test['clickDate'] = test['clickTime'].apply(lambda x : x/10000)

In [4]:
train['clickDate'] = train['clickTime'].apply(lambda x : x/10000)

In [7]:
train  = train[(train.clickDate == 24)
              | (train.clickDate == 17)]

In [11]:
train  = train[train.clickDate != 30]

In [12]:
train['source']= 'train'
test['source'] = 'test'
data=pd.concat([train, test],ignore_index=True)
data.shape

(918284, 11)

In [13]:
ad = pd.read_csv('../dataset/pre/ad.csv')
#app_categories = pd.read_csv('../dataset/pre/app_categories.csv')
position = pd.read_csv('../dataset/pre/position.csv')
user = pd.read_csv('../dataset/pre/user.csv')
#user_app_actions = pd.read_csv('../dataset/pre/user_app_actions.csv')
#user_installedapps = pd.read_csv('../dataset/pre/user_installedapps.csv')

-  特征重要性观察中发现，creativeID 和 positionID 是重要特征，于是单独列出，稀释比例
-  随机森林中，用户所占比重过大，考虑稀释

In [40]:
data = data.merge(ad ,on = 'creativeID')

data = data.merge(position, on = 'positionID')

data = data.merge(user, on = 'userID')

In [14]:
train = data[data.source == 'train']
train.drop('source',axis=1,inplace=True)
test = data[data.source == 'test']
test.drop('source',axis=1,inplace=True)

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [16]:
feature = [x for x in train.columns if x not in ['label','conversionTime']]
target = 'label'

In [17]:
model = RandomForestClassifier()

In [18]:
model.fit(train[feature].fillna(-1),train[target])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [19]:
feature

['clickDate',
 'clickTime',
 'connectionType',
 'creativeID',
 'instanceID',
 'positionID',
 'telecomsOperator',
 'userID']

In [21]:
model.feature_importances_

array([ 0.00172931,  0.36519427,  0.00469371,  0.09912568,  0.        ,
        0.112893  ,  0.01035398,  0.40601005])

In [22]:
predict1 = model.predict(test[feature].fillna(-1))
out1 = model.predict_proba(test[feature].fillna(-1))

In [23]:
pd.DataFrame(predict1)[0].value_counts()

0    337919
1       570
Name: 0, dtype: int64

In [24]:
model2 = xgb.XGBClassifier()

In [25]:
model2.fit(train[feature],train[target])

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [26]:
feature

['clickDate',
 'clickTime',
 'connectionType',
 'creativeID',
 'instanceID',
 'positionID',
 'telecomsOperator',
 'userID']

In [27]:
model2.feature_importances_

array([ 0.        ,  0.06916427,  0.11527377,  0.32420748,  0.        ,
        0.4668588 ,  0.00288184,  0.02161383], dtype=float32)

In [28]:
predict2 = model2.predict(test[feature])
out2 = model2.predict_proba(test[feature])

In [29]:
pd.DataFrame(predict2)[0].value_counts()

0    338489
Name: 0, dtype: int64

In [30]:
out1[:,1]

array([ 0. ,  0.1,  0.1, ...,  0. ,  0. ,  0. ])

In [31]:
out2[:,1]

array([ 0.01477208,  0.01931296,  0.00567335, ...,  0.03020852,
        0.00830845,  0.0052004 ], dtype=float32)

In [33]:
predict = out1[:,1]*0.1 + out2[:,1]*0.9

In [39]:
predict = out2[:,1]

In [40]:
submission = pd.DataFrame(test['instanceID'])

In [41]:
submission['instanceID'] = submission['instanceID'].astype(int)

In [42]:
submission = submission.sort_values('instanceID')

In [43]:
submission['pro'] = predict

In [44]:
submission.to_csv('../output/submission.csv',index=False)